In [1]:
%config Completer.use_jedi = False
import sys
import warnings
warnings.filterwarnings('ignore')
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import scipy.io
import time
import threading
import gc
import copy

sys.path.append("/end/home/rh2310/morpho_repo/turing_codebase")
from turing.utils import *
import turing.pinns as tu
from turing.loss_functions import *

GPU = True
import os

if GPU:
    txt_device = 'gpu:0'
else:
    txt_device = 'cpu:0'    
    os.environ["CUDA_VISIBLE_DEVICES"]="-1"

2022-11-01 10:38:13.514278: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1900] Ignoring visible gpu device (device: 0, name: Quadro K2000, pci bus id: 0000:05:00.0, compute capability: 3.0) with Cuda compute capability 3.0. The minimum required Cuda capability is 3.5.


# Load data from the output of Brusselator model

In [11]:
import os
data_path = os.path.abspath("../turing_Brusselator.npy")
with open(data_path, 'rb') as f:
    data = np.load(f)
    
data_path = os.path.abspath("../turing_t_Brusselator.npy")
with open(data_path, 'rb') as f:
    t_star = np.load(f) 
    
# Thining the dataset
t_star = t_star[20::16]
data = data[..., 20::16]

T = t_star.shape[0]    
    
L = 1.0
x_size = data.shape[1]
y_size = data.shape[2]
N = x_size*y_size

model_params = {'training_data_size': T*N,#T*32,
                'pde_data_size': (T*N)//(32),
                'boundary_data_size':((x_size + y_size)*T)//(8)}

dataset = create_dataset(data, t_star, N, T, L, **model_params)
lb = dataset['lb']
ub = dataset['ub']
obs_X = dataset['obs_input']
obs_Y = dataset['obs_output']
pde_X = dataset['pde']


In [3]:
T

15

# Run Koch-Meinhardt's ASDM model with given diffusion constants and correct space-time scalse to find other parameters

In [3]:
dataset = TINN_Single_Sim_Dataset(os.path.abspath("../../test/outputs_Brusselator_100_by_100/Brusselator_1"),
                                 "Brusselator_1",
                                 thining_start=-2)

In [3]:
#layers = [3, 64, 64, 64, 64, 2]
#layers = [3, 128, 128, 128, 128, 2]
layers = [3, 10, 10, 10, 2]

pinn = tu.NN(layers, dataset.lb, dataset.ub, dtype=tf.float64)
pde_loss = ASDM(D_u=PDE_Parameter("D_u", Parameter_Type.CONSTANT, 1,dtype=pinn.dtype),
                D_v=PDE_Parameter("D_v", Parameter_Type.CONSTANT, 1,dtype=pinn.dtype),
                sigma_u=PDE_Parameter("sigma_u", Parameter_Type.CONSTANT, 1,dtype=pinn.dtype),
                sigma_v=PDE_Parameter("sigma_v", Parameter_Type.CONSTANT, 1,dtype=pinn.dtype),
                rho_u=PDE_Parameter("rho_u", Parameter_Type.CONSTANT, 1,dtype=pinn.dtype),
                rho_v=PDE_Parameter("rho_v", Parameter_Type.CONSTANT, 1,dtype=pinn.dtype),
                mu_u=PDE_Parameter("mu_u", Parameter_Type.CONSTANT, 1,dtype=pinn.dtype),
                kappa_u=PDE_Parameter("kappa_u", Parameter_Type.CONSTANT, 1,dtype=pinn.dtype),
                )
#pde_non_zero = Non_zero_params("ASDM", parameters=[pde_loss.D_u, pde_loss.D_v], print_precision=".10f")

l2_norm = L2()
l_inf_norm = L_Inf()

In [4]:
model = tu.TINN(pinn, 
                pde_loss, 
                loss = l2_norm + l_inf_norm,
                #non_zero_loss=pde_non_zero, 
                #optimizer=keras.optimizers.Adam(learning_rate=0.0001), 
                alpha = 0.5,
                print_precision=".10f")

In [5]:
results = model.train(epochs=10,
                      batch_size=dataset.x_size//8,
                      dataset=dataset,
                      print_interval=1,
                      stop_threshold=1e-5,
                      sample_losses=True, 
                      sample_parameters=True,
                      sample_regularisations=True,
                      sample_gradients=True,
                      regularise=True
                     )


Start of epoch 0


2022-11-01 10:32:29.549718: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


Training observations acc over epoch: 4.3264904022
total loss: 21.2878979893, total regularisd loss (sum of batches): 170.3031839142
obs u loss: 14.3203735943, obs v loss: 2.9855875070
pde u loss: 2.3386415579, pde v loss: 1.6432953300
lambda obs u: 1.0000000000, lambda obs v: 1.0000000000
lambda pde u: 1.0000000000, lambda pde v: 1.0000000000

Time taken: 19.78s

Start of epoch 1
Training observations acc over epoch: 4.2709894180
total loss: 20.5487641914, total regularisd loss (sum of batches): 164.3901135310
obs u loss: 14.1580784328, obs v loss: 2.9258797806
pde u loss: 1.9944246558, pde v loss: 1.4703813222
lambda obs u: 2.3048627360, lambda obs v: 0.6158554393
lambda pde u: 0.5423605430, lambda pde v: 0.5369212817

Time taken: 5.64s

Start of epoch 2
Training observations acc over epoch: 4.2173547745
total loss: 19.8854572975, total regularisd loss (sum of batches): 285.2081070923
obs u loss: 13.9923921682, obs v loss: 2.8770265083
pde u loss: 1.7028680232, pde v loss: 1.31317059

In [4]:
model = tu.TINN.restore(".", "ttt_model", dataset)

2022-11-01 10:38:41.746544: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


In [5]:
results = model.train(epochs=10,
                      batch_size=dataset.x_size//8,
                      dataset=dataset,
                      print_interval=1,
                      stop_threshold=1e-5,
                      sample_losses=True, 
                      sample_parameters=True,
                      sample_regularisations=True,
                      sample_gradients=True,
                      regularise=True
                     )


Start of epoch 0
Training observations acc over epoch: 3.6630735397
total loss: 16.3038350046, total regularisd loss (sum of batches): 350.4495756340
obs u loss: 11.8244021490, obs v loss: 2.8278921080
pde u loss: 0.6747090917, pde v loss: 0.9768316559
lambda obs u: 3.6311555915, lambda obs v: 0.2805856632
lambda pde u: 0.0321953584, lambda pde v: 0.0560633869

Time taken: 16.21s

Start of epoch 1
Training observations acc over epoch: 3.5662655830
total loss: 15.8309160803, total regularisd loss (sum of batches): 339.7509331592
obs u loss: 11.4584784700, obs v loss: 2.8065849738
pde u loss: 0.5834595862, pde v loss: 0.9823930503
lambda obs u: 3.6171997357, lambda obs v: 0.2971298701
lambda pde u: 0.0251147548, lambda pde v: 0.0605556393

Time taken: 5.03s

Start of epoch 2
Training observations acc over epoch: 3.4627938271
total loss: 15.3313658305, total regularisd loss (sum of batches): 327.5783402234
obs u loss: 11.0719076388, obs v loss: 2.7792672389
pde u loss: 0.4922586038, pde 

In [15]:
weight_values = model.optimizer.get_weights()

In [16]:
len(weight_values)

33

In [76]:
data_path = pathlib.PurePath(os.path.abspath("../../test/outputs_tinn_Brusselator_100_by_100_noise/Brusselator_000_noise/Brusselator_000_noise_model_optimizer.pkl"))
with open(data_path, "rb") as f:
    data = np.load(f, allow_pickle=True)

In [115]:
len(data)

25

In [114]:
len(weight_values)

49

In [92]:
with open(os.path.abspath("./tttt.npy"), "wb") as f:
    pickle.dump(weight_values, f)

In [93]:
with open(os.path.abspath("./tttt.npy"), "rb") as f:
    weight_values2 = pickle.load(f)

In [112]:
weight_values[7].shape

(1, 128)

In [113]:
weight_values2[7].shape

(1, 128)

In [6]:
model.save(".", "ttt_model")

In [9]:
model.optimizer.from_config(model.optimizer.get_config())

In [31]:
import pathlib
path = pathlib.PurePath(".").joinpath("ttt_model")
# asset = tf.saved_model.load(str(path))
with open(f"{str(path)}.pkl", "rb") as f:
    model2 = pickle.load(f)
with open(f"{str(path)}_optimizer.pkl", "rb") as f:
    weight_values2 = pickle.load(f)
with open(f"{str(path)}_optimizer_config.pkl", "rb") as f:
    conf2 = pickle.load(f)


In [32]:
 model2.optimizer = keras.optimizers.Adam(learning_rate=5e-4).from_config(conf2)

In [25]:
trainables = model2.pinn.trainable_variables + model2.pde_residual.trainables()
for extra_loss in model2.extra_loss:
    trainables += extra_loss.trainables()

if model2.non_zero_loss is not None:
    non_zero_params = model2.non_zero_loss.parameters
# dummy zero gradients
zero_grads = [tf.zeros_like(w) for w in trainables]
# save current state of variables
saved_vars = [tf.identity(w) for w in trainables]



if model2.non_zero_loss is not None:
    # dummy zero gradients
    zero_grads_params = [tf.zeros_like(w) for w in model2.non_zero_loss.parameters]
    # save current state of variables
    saved_vars_params = [tf.identity(w) for w in model2.non_zero_loss.parameters]

# Apply gradients which don't do nothing with Adam
model2.optimizer.apply_gradients(zip(zero_grads, trainables))
if model2.non_zero_loss is not None:
    model2.optimizer.apply_gradients(zip(zero_grads_params, model2.non_zero_loss.parameters))

# Reload variables
[x.assign(y) for x, y in zip(trainables, saved_vars)]
if model2.non_zero_loss is not None:
    [x.assign(y) for x, y in zip(model2.non_zero_loss.parameters, saved_vars_params)]

In [33]:
len(trainables)

8

In [34]:
len(weight_values2)

33

In [44]:
X, Y = next(iter(dataset.take(1)))#).numpy()